In [3]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Dash setup with proper assets path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🍽️ Food & Produce Carousel"

# 📦 Corrected category paths
CATEGORY_MAP = {
    "Prepared food": "../assets/object/food",
    "Fruit": "../assets/object/fruit",
    "Vegetable": "../assets/object/vegetable"
}

# 📦 List items per category
category_items = {
    label: sorted([
        item for item in os.listdir(path)
        if os.path.isdir(os.path.join(path, item))
    ])
    for label, path in CATEGORY_MAP.items()
}

# 📦 {(category, item): [list of image paths]}
image_lookup = {}
for category, path in CATEGORY_MAP.items():
    for item in category_items[category]:
        full_path = os.path.join(path, item)
        image_lookup[(category, item)] = sorted([
            f"{full_path}/{img}"
            for img in os.listdir(full_path)
            if img.lower().endswith((".jpg", ".jpeg", ".png"))
        ])

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🍽️ Food & Produce Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select category:", style={"fontWeight": "bold", "marginBottom": "6px"}),
        dcc.Dropdown(
            id='category-dropdown',
            options=[{"label": cat, "value": cat} for cat in CATEGORY_MAP],
            value="Prepared food",
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "16px"}),

    html.Div(id='item-dropdown-container', style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Update item dropdown based on category
@app.callback(
    Output('item-dropdown-container', 'children'),
    Input('category-dropdown', 'value')
)
def update_item_dropdown(category):
    items = category_items.get(category, [])
    return html.Div([
        html.Label("Select item:", style={"fontWeight": "bold", "marginBottom": "6px"}),
        dcc.Dropdown(
            id='item-dropdown',
            options=[{"label": item.replace("_", " ").capitalize(), "value": item} for item in items],
            value=items[0] if items else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ])

# 🔁 Carousel content
@app.callback(
    Output('carousel-container', 'children'),
    Input('category-dropdown', 'value'),
    Input('item-dropdown', 'value')
)
def update_carousel(category, item):
    images = image_lookup.get((category, item), [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "20px", "padding": "5px 10px", "marginRight": "20px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "20px", "padding": "5px 10px", "marginLeft": "20px"}),
        ], style={"textAlign": "center", "marginBottom": "12px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "13px", "fontStyle": "italic", "fontFamily": "Georgia, serif"})
    ])

# 🔁 Navigation buttons
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Launch (change port if needed)
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8802)
